In [1]:
# import common packages
import numpy as np

import qiskit
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit

# lib from Qiskit AQUA Chemistry
from qiskit_aqua_chemistry import FermionicOperator

# lib from optimizer and algorithm
from qiskit_aqua.operator import Operator
from qiskit_aqua import (get_algorithm_instance, get_optimizer_instance, get_variational_form_instance)

# lib for driver
from qiskit_aqua_chemistry.drivers import ConfigurationManager
from collections import OrderedDict

In [2]:
# using driver to get fermionic Hamiltonian
# PyQuante example
cfg_mgr = ConfigurationManager()
pyquante_cfg = OrderedDict([('atoms', 'H .0 .0 .0; H .0 .0 0.735'), ('units', 'Angstrom'), ('charge', 0), ('multiplicity', 1), ('basis', 'sto3g')])
section = {}
section['properties'] = pyquante_cfg
driver = cfg_mgr.get_driver_instance('PYQUANTE')
molecule = driver.run(section)
h1 = molecule._one_body_integrals
h2 = molecule._two_body_integrals

In [3]:
# convert from fermionic hamiltonian to qubit hamiltonian
ferOp = FermionicOperator(h1=h1, h2=h2)
qubitOp_jw = ferOp.mapping(map_type='JORDAN_WIGNER', threshold=0.00000001)
qubitOp_pa = ferOp.mapping(map_type='PARITY', threshold=0.00000001)
qubitOp_bi = ferOp.mapping(map_type='BRAVYI_KITAEV', threshold=0.00000001)

In [4]:
# print out qubit hamiltonian in Pauli terms and exact solution

qubitOp_jw.convert('paulis','matrix')
qubitOp_jw.chop(10**-10)

print(qubitOp_jw.print_operators())
print(qubitOp_jw.matrix)

# Using exact eigensolver to get the smallest eigenvalue
exact_eigensolver = get_algorithm_instance('ExactEigensolver')
exact_eigensolver.init_args(qubitOp_jw, k=1)
ret = exact_eigensolver.run()
print('The exact ground state energy is: {}'.format(ret['energy']))    

IIII	(-0.8105479862761009+0j)
ZIII	(0.17218394273085644+0j)
IZII	(-0.2257535025154054+0j)
IIZI	(0.1721839427308564+0j)
IIIZ	(-0.2257535025154054+0j)
IZZI	(0.16614543338049353+0j)
YYYY	(0.045232799794893475+0j)
YYXX	(0.045232799794893475+0j)
XXYY	(0.045232799794893475+0j)
XXXX	(0.045232799794893475+0j)
ZZII	(0.12091263358560006+0j)
ZIIZ	(0.16614543338049353+0j)
ZIZI	(0.16892754048859018+0j)
IZIZ	(0.174643431424422+0j)
IIZZ	(0.12091263358560006+0j)

  (1, 1)	(-1.2563391003710798+0j)
  (2, 2)	(-0.4718959917502202+0j)
  (3, 3)	(-1.2445845577788999+0j)
  (4, 4)	(-1.2563391003710798+0j)
  (5, 5)	(-1.8369680387877996+0j)
  (5, 10)	(0.1809311991795739+0j)
  (6, 6)	(-1.0636533585993264+0j)
  (6, 9)	(0.1809311991795739+0j)
  (7, 7)	(-1.1606317626736458+0j)
  (8, 8)	(-0.47189599175021985+0j)
  (9, 6)	(0.1809311991795739+0j)
  (9, 9)	(-1.0636533585993264+0j)
  (10, 5)	(0.1809311991795739+0j)
  (10, 10)	(-0.2452182578027522+0j)
  (11, 11)	(-0.35332509030945825+0j)
  (12, 12)	(-1.2445845577788999+0j

In [5]:
# setup VQE 
# setup optimizer, use L_BFGS_B optimizer for example
lbfgs = get_optimizer_instance('L_BFGS_B')
lbfgs.set_options(maxfun=1000, factr=10, iprint=10)
# setup variation form generator (generate trial circuits for VQE)
var_form = get_variational_form_instance('RY')
var_form.init_args(qubitOp_jw.num_qubits, 3, entangler_map = {0: [1], 1:[2], 2:[3]})

# setup VQE with operator, variation form, and optimzer
vqe_algorithm = get_algorithm_instance('VQE')
vqe_algorithm.setup_quantum_backend()
vqe_algorithm.init_args(qubitOp_jw, 'matrix', var_form, lbfgs)
results = vqe_algorithm.run()
print("Minimum value: {}".format(results['eigvals'][0]))
print("Parameters: {}".format(results['opt_params']))

Minimum value: -1.8572750764933232
Parameters: [-2.11901756e+00  1.41323639e+00  2.51387300e+00 -3.13978348e+00
 -2.82577526e+00  2.70457462e+00  1.20214170e-01  1.07093537e+00
  1.09731868e+00 -1.90329651e+00  2.16009766e+00  1.56901864e+00
  3.13125831e+00 -3.11613989e-04  2.32391406e+00  2.36493298e+00]
